# TP3 - Views, Updates and Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP2.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [65]:
import sqlite3

In [66]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [67]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
clients:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
producers:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
sales:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: REGION(TEXT)
  5: QTE(NUM)
wines:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


Again, we will use **`%%sql`** magic for our queries

In [68]:
%load_ext sql
%sql sqlite:///wine.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @wine.db'

Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP2

**wines**

|*Attribute*| *Description*                  |
|:----------|:-------------------------------|
| NV [PK]   | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |


**production**

|*Attribute*| *Description*                  |
|:----------|:-------------------------------|
| NV [PK]   | Wine number                    |
| NP [PK]   | Producer number                |
| QTE       | Number of bottles harvested    |


**producers**

|*Attribute*| *Description*                  |
|:----------|:-------------------------------|
| NP [PK]   | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

**sales**

|*Attribute* | *Description*                                          |
|:-----------|:-------------------------------------------------------|
| NV [PK]    | Wine number                                            |
| NB [PK]    | Client (buveur) number                                 |
| DATES [PK] | Buying date                                            |
| LIEU [PK]  | Place where the wine was sold                          |
| REGION     | Administrative Region (different to production region) |
| QTE        | Number of bottles bought                               |

**clients**

|*Attribute*| *Description*                                          |
|:----------|:-------------------------------------------------------|
| NB [PK]   | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |


In [69]:
%%sql DROP TABLE IF EXISTS wines;
CREATE TABLE wines AS
SELECT DISTINCT NV, CRU, DEGRE, MILL
FROM MASTER1
WHERE NV IS NOT NULL;
SELECT *
FROM wines
LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
2,Julienas,11.3,1974
3,Savigny les Beaunes,12.1,1978
4,Mercurey,10.9,1980
5,Pommard,11.7,1976
6,Mercurey,11.2,1981
7,Grands Echezeaux,11.7,1968
8,Cotes de Beaune Villages,12.3,1975
9,Chapelle Chambertin,11.9,1973
10,Beaujolais Villages,11.8,1979


In [70]:
%%sql DROP TABLE IF EXISTS production;
CREATE TABLE production AS
SELECT DISTINCT NP, NV, QTE
FROM MASTER1
WHERE NP IS NOT NULL AND NV IS NOT NULL;
SELECT *
FROM production
LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NP,NV,QTE
1,1,300
73,1,1
5,2,100
1,3,400
10,4,35
30,5,46
42,6,300
98,7,60
90,8,12
98,10,100


In [71]:
%%sql DROP TABLE IF EXISTS producers;
CREATE TABLE producers AS
SELECT DISTINCT NP, NOM, PRENOM, REGION
FROM MASTER1
WHERE NP IS NOT NULL;
SELECT *
FROM producers
LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NP,NOM,PRENOM,REGION
3,Six,Paul,Alsace
6,Marmagne,Bernard,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
16,Barbin,Bernard,Bourgogne
17,Faiveley,Guy,Bourgogne
18,Tramier,Jean,Bourgogne
19,Dupaquier,Roger,Bourgogne
20,Lamy,Jean,Bourgogne
21,Cornu,Edmond,Bourgogne
26,Violot,Gilbert,Bourgogne


In [72]:
%%sql DROP TABLE IF EXISTS sales;
CREATE TABLE sales AS
SELECT DISTINCT NB, NV, DATES, LIEU, REGION, QTE 
FROM MASTER2
WHERE NB IS NOT NULL AND NV IS NOT NULL AND DATES IS NOT NULL AND LIEU IS NOT NULL;
SELECT *
FROM sales
LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NV,DATES,LIEU,REGION,QTE
2,1,1977-11-02,BORDEAUX,NOUVELLE-AQUITAINE,33
44,1,2015-10-16,PARIS,ÎLE-DE-FRANCE,1
45,1,1983-12-31,RENNES,BRETAGNE,1
48,2,1983-12-25,LYON,AUVERGNE-RHÔNE-ALPES,2
7,3,1978-11-01,NICE,PROVENCE-ALPES-CÔTE D'AZUR,6
49,3,1987-12-24,MERCUREY,BOURGOGNE-FRANCHE-COMTÉ,5
8,4,1982-11-05,TOULOUSE,OCCITANIE,12
44,4,2015-10-16,PARIS,ÎLE-DE-FRANCE,6
50,4,1981-06-29,SENS,BOURGOGNE-FRANCHE-COMTÉ,4
2,5,1984-11-04,MERCUREY,BOURGOGNE-FRANCHE-COMTÉ,50


In [73]:
%%sql DROP TABLE IF EXISTS clients;
CREATE TABLE clients AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE
FROM MASTER2
WHERE NB IS NOT NULL;
SELECT *
FROM clients
LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


In [74]:
printSchema(conn)

MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
clients:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
producers:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
sales:
  0: NB(NUM)
  1: NV(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: REGION(TEXT)
  5: QTE(NUM)
wines:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [75]:
%%sql DROP VIEW IF EXISTS bons_buveurs;
CREATE VIEW bons_buveurs AS
SELECT * 
FROM clients
WHERE TYPE = 'gros' OR TYPE = 'moyen';
SELECT *
FROM bons_buveurs
LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros
32,Eluard,Paul,moyen
35,Fromentin,Eugene,gros
39,Montesquieu,None,gros
42,Goethe,Johann Wolfgang von,moyen
43,Musset,Alfred de,gros


In [76]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [77]:
%%sql DROP VIEW IF EXISTS buveurs_asec;
CREATE VIEW buveurs_asec AS
SELECT * 
FROM clients
WHERE clients.NB NOT IN (SELECT sales.NB FROM sales)
GROUP BY clients.NB ;

 * sqlite:///wine.db
Done.
Done.


[]

In [78]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [79]:
%%sql DROP VIEW IF EXISTS buveurs_achats;
CREATE VIEW buveurs_achats AS
SELECT * 
FROM clients
WHERE clients.NB IN (SELECT sales.NB FROM sales)
GROUP BY clients.NB ;

 * sqlite:///wine.db
Done.
Done.


[]

In [80]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,petit
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
9,Ajar,Emile,petit
10,Andersen,Yann,gros


#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [81]:
%%sql DROP VIEW IF EXISTS q83pl;
CREATE VIEW q83pl AS
SELECT LIEU, CRU, SUM(QTE) AS QTE_BUE
FROM wines
JOIN sales    ON           sales.NV = wines.NV 
WHERE DATES LIKE '1983%' 
GROUP BY LIEU, CRU;
SELECT *
FROM q83pl
LIMIT 10;


 * sqlite:///wine.db
Done.
Done.
Done.


LIEU,CRU,QTE_BUE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


In [82]:
# Test
%sql SELECT * FROM q83pl;

 * sqlite:///wine.db
Done.


LIEU,CRU,QTE_BUE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

**Answer:** Sorting with ascending order over the attribute QTE can be done either during the creation of the view or after the creation, while we are calling it. It is way **more recommended not to put ORDER BY in views** because it is not efficient in computing time. It is usually preferable to keep the view simple. Indeed, the use of ORDER BY is usually more preferable in the queries that reference them. 

However, It would be possible to sort the view by creating a new identical view but sorted, depending on the use and the context of the application.

Views are not meant to dictate presentation order; if we expect queries against a view from presenting the data in a predictable order, we need to add those ORDER BY clauses to our outer queries. So, generally, having ORDER BY inside the view is not going to work in all cases, and is going to be confusing for people reading or maintaining the code.

**Not recommended :**

In [83]:
%%sql DROP VIEW IF EXISTS q83pl;
CREATE VIEW q83pl AS
SELECT LIEU, CRU, SUM(QTE) AS QTE_BUE
FROM wines
JOIN sales    ON           sales.NV = wines.NV 
WHERE DATES LIKE '1983%' 
GROUP BY LIEU, CRU 
ORDER BY QTE_BUE ASC;
SELECT *
FROM q83pl
LIMIT 10;

 * sqlite:///wine.db
Done.
Done.
Done.


LIEU,CRU,QTE_BUE
PARIS,Coteaux du Tricastin,1
RENNES,Mercurey,1
LYON,Julienas,2
CAEN,Seyssel,3
PARIS,Pouilly Vinzelles,3
PARIS,Beaujolais Primeur,4
LILLE,Pommard,5
LYON,Beaujolais Villages,10
ROCQUENCOURT,Saint Amour,80
ROCQUENCOURT,Beaujolais Villages,260


In [84]:
%%sql DROP VIEW IF EXISTS q83pl;
CREATE VIEW q83pl AS
SELECT LIEU, CRU, SUM(QTE) AS QTE_BUE
FROM wines
JOIN sales    ON           sales.NV = wines.NV 
WHERE DATES LIKE '1983%' 
GROUP BY LIEU, CRU;

 * sqlite:///wine.db
Done.
Done.


[]

**Recommended :**

In [85]:
# sorting over QTE
%sql SELECT *  FROM q83pl ORDER BY QTE_BUE ASC;


 * sqlite:///wine.db
Done.


LIEU,CRU,QTE_BUE
PARIS,Coteaux du Tricastin,1
RENNES,Mercurey,1
LYON,Julienas,2
CAEN,Seyssel,3
PARIS,Pouilly Vinzelles,3
PARIS,Beaujolais Primeur,4
LILLE,Pommard,5
LYON,Beaujolais Villages,10
ROCQUENCOURT,Saint Amour,80
ROCQUENCOURT,Beaujolais Villages,260


___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [86]:
%%sql DROP TABLE IF EXISTS RBB;
CREATE TABLE RBB AS
SELECT *
FROM bons_buveurs
ORDER by bons_buveurs.NB;

 * sqlite:///wine.db
Done.
Done.


[]

In [87]:
# Test
%sql SELECT * FROM RBB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may differ from the one in your table)

In [88]:
%%sql
SELECT clients.NB, clients.NOM, clients.TYPE, SUM(sales.QTE) AS  total_of_quantities_bought
FROM clients
    JOIN sales ON clients.NB = sales.NB
WHERE clients.TYPE IN ('petit', 'moyen')
GROUP BY (clients.NB)
HAVING total_of_quantities_bought > 100

 * sqlite:///wine.db
Done.


NB,NOM,TYPE,total_of_quantities_bought
2,Artaud,moyen,583
5,Audiberti,petit,113
9,Ajar,petit,140
44,Gide,petit,171


Update instances

In [89]:
%%sql
SELECT * FROM clients
    JOIN sales ON clients.NB = sales.NB
WHERE clients.TYPE IN ('petit', 'moyen')
GROUP BY clients.NB
HAVING SUM(sales.QTE)>100

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,NB_1,NV,DATES,LIEU,REGION,QTE
2,Artaud,Antonin,moyen,2,1,1977-11-02,BORDEAUX,NOUVELLE-AQUITAINE,33
5,Audiberti,Jacques,petit,5,9,1982-03-26,LYON,AUVERGNE-RHÔNE-ALPES,20
9,Ajar,Emile,petit,9,8,1979-11-01,LILLE,HAUTS-DE-FRANCE,10
44,Gide,Andre,petit,44,1,2015-10-16,PARIS,ÎLE-DE-FRANCE,1


In [90]:
%%sql
UPDATE clients
SET TYPE = 'gros'
WHERE clients.NB IN (SELECT clients.NB FROM clients
                    JOIN sales ON clients.NB = sales.NB
                    WHERE clients.TYPE IN ('petit', 'moyen')
                    GROUP BY clients.NB
                    HAVING SUM(sales.QTE)>100);

 * sqlite:///wine.db
4 rows affected.


[]

In [91]:
%%sql
SELECT * FROM RBB
WHERE RBB.NB IN (2, 5, 9, 44);

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen


#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

**Answer:** <br/>

As we identified the clients.NB (= 2, 5, 9, 44) for which the clients.TYPE was changed to 'gros' in the **original TABLE**, we notice that the **TABLE RBB** created from the **VIEW bon_buveurs** (itself created from the **original TABLE clients**) still displays for RBB.NB = 2 :
- RBB.TYPE = 'moyen' and not RBB.TYPE = 'gros', 

whereas we can observe in the following that **the VIEW bon_buveurs is automatically well updated** :

- RBB.TYPE = 'gros'. <br/>

Indeed, __a view is a request script__ and it is always updated with the corresponding TABLE if this last one is changed.  

In [94]:
%%sql
SELECT * FROM bons_buveurs
WHERE bons_buveurs.NB IN (2, 5, 9, 44);

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,gros
44,Gide,Andre,gros
9,Ajar,Emile,gros
5,Audiberti,Jacques,gros


#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [97]:
%%sql
DROP TABLE IF EXISTS RBA;
CREATE TABLE RBA AS
SELECT * FROM buveurs_asec;

 * sqlite:///wine.db
Done.
Done.


[]

In [98]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [99]:
%%sql
INSERT INTO RBA VALUES (101, 'First_Name', 'Last_Name', 'gros');

 * sqlite:///wine.db
1 rows affected.


[]

In [100]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain

**Answer:**<br/>

The TABLE RBA was updated with a new client.NB = 101. So logically, we observe that **the VIEW buveurs_asec created as a request on the original TABLE clients** remains unchanged (without client.NB = 101). <br/>

Indeed, it would have been automatically changed **only if** :
- it was created from TABLE RBA or 
- if the TABLE clients was changed.

In [102]:
%%sql
SELECT * FROM buveurs_asec;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

We decide to use 4 tables : <br>
- **users** : user_id (Primary Key), name, nickname, email, date_of_birth, address_id <br>
- **address** : address_id (Primary Key), street_city, state, country, postal_code <br>
- **posts** : post_id (Primary Key), text, date, attachment, original_post_id <br>
- **friendships** : user_id (Primary Key), friend_id (Primary Key), following_flag  <br>

<img src="schema_ER_TP3_6.png">


#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.

Basically, in order to retrive the posts to display when a user logs in, we need to create a view including specific interesting attributes described in the following such as the **post date**, and the **original_post_id** that allows computing the sum **number of replies**. Also we would **count the number of friends** for each user, and add a **'following' flag** in the friendship table. The created view would be sorted using the described attributes. <br>

1- In order to display the most interesting posts, we first consider **the date already included in the post** table.<br> 
So basically, sorting the posts per publication date is the first thing to do. <br>

2- Secondly, another thing is to **sort the post per number of replies (ReplyNb).** Indeed we would create a view that count the post_id grouped by original_post_id. Note that if post_id = original_post_id (ReplyNb=0), the post is the original and need to appear on the top.
So we could sort the post by ascending ReplyNb and display in priority 
- ReplyNb = 0 (original post) and 
- Descending_order(ReplyNb) (most popular post)<br>
So we use the view :<br>
- **replies** : post_id (Primary Key), **reply_nb** <br>

3- Another way to sort the posts to display them, when a user logs in, would be to add in the friendship table a **following_flag** in order that a user could activate or deactivate a friend to denote its interest in some particular friend posts. Note that for each frienship, both user_id and friend_id had to be doubled and the following_flag induce that one guy can be friend with another guy but each of them are not supposed to be follower the other one. <br>

4- Finally, another sort could consider the popular users by **counting the number of friends** for each user. Someone having more friend would be more traited like an "influencer" so that its posts would appear first in the news feeds. <br>
